# 03 – Hyperparameter Tuning with Bayesian Optimization
Use bayesian-optimization to tune a RandomForestClassifier.

In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from bayes_opt import BayesianOptimization

df = pd.read_csv(Path('../data/raw/creditcard.csv'))
X = df.drop(columns=['Class'])
y = df['Class']

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

cols_to_scale = ['Amount'] + (['Time'] if 'Time' in X.columns else [])
scaler = StandardScaler()
X_train[cols_to_scale] = scaler.fit_transform(X_train[cols_to_scale])
X_valid[cols_to_scale] = scaler.transform(X_valid[cols_to_scale])

def cv_auc(n_estimators, max_depth, max_features, min_samples_split, min_samples_leaf):
    from sklearn.ensemble import RandomForestClassifier
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    min_samples_split = int(min_samples_split)
    min_samples_leaf = int(min_samples_leaf)
    max_features = max(min(max_features, 1.0), 0.1)
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth if max_depth>0 else None,
        max_features=max_features,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        class_weight='balanced_subsample',
        random_state=42,
        n_jobs=-1
    )
    clf.fit(X_train, y_train)
    proba = clf.predict_proba(X_valid)[:,1]
    return roc_auc_score(y_valid, proba)

pbounds = {
    'n_estimators': (100, 600),
    'max_depth': (4, 20),
    'max_features': (0.2, 1.0),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 10),
}

optimizer = BayesianOptimization(
    f=cv_auc,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

optimizer.maximize(init_points=8, n_iter=20)
print('Best params:', optimizer.max)

|   iter    |  target   | n_esti... | max_depth | max_fe... | min_sa... | min_sa... |
-------------------------------------------------------------------------------------
| 2         | 0.9737882 | 177.99726 | 4.9293377 | 0.8929409 | 12.820070 | 7.3726532 |
| 3         | 0.9609985 | 110.29224 | 19.518557 | 0.8659541 | 5.8221039 | 2.6364247 |
| 4         | 0.9750126 | 191.70225 | 8.8678758 | 0.6198051 | 9.7750103 | 3.6210622 |
| 5         | 0.9788734 | 405.92644 | 6.2319017 | 0.4337157 | 8.5945131 | 5.1046298 |


In [ ]:
# Show the single best result (score + params)
print("Best result:", optimizer.max)

# Extract only the best params
best_params = optimizer.max["params"]

# Cast to the correct types for sklearn
best_params = {
    "n_estimators": int(best_params["n_estimators"]),
    "max_depth": int(best_params["max_depth"]) if int(best_params["max_depth"]) > 0 else None,
    "max_features": float(best_params["max_features"]),     # stays float in (0,1]
    "min_samples_split": int(best_params["min_samples_split"]),
    "min_samples_leaf": int(best_params["min_samples_leaf"]),
}
best_params